In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.applications.densenet import DenseNet121
from keras import regularizers

# Входные данные
input_shape = (128, 128, 3)
inputs = Input(input_shape)

# Подготовка предобученной модели DenseNet121
base_model = DenseNet121(weights='imagenet', include_top=False,
                         input_tensor=inputs, input_shape=input_shape)

# Добавление головы классификатора
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='sigmoid')(x)

# Объединение входов и выходов в единую модель
model = Model(inputs=inputs, outputs=predictions)

# Замораживание слоев базовой модели
for layer in base_model.layers:
    layer.trainable = False
    
# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Вывод информации о модели
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_14 (ZeroPadding  (None, 134, 134, 3)  0          ['input_8[0][0]']                
 2D)                                                                                              
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 64, 64)   9408        ['zero_padding2d_14[0][0]']      
                                                                                            

In [12]:
import numpy as np
from keras.utils import to_categorical

x_train= np.load('drive/MyDrive/cv_pnem/x_train.npy')
y_train = np.load('drive/MyDrive/cv_pnem/y_train.npy')
x_test= np.load('drive/MyDrive/cv_pnem/x_test.npy')
y_test = np.load('drive/MyDrive/cv_pnem/y_test.npy')
x_train = x_train / 255
x_test = x_test / 255

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [13]:
#нахождение предполагаемого глобального минимума
min_acc_train = 0.97
min_acc_test = 0.87
while True: 
  input_shape = (128, 128, 3)
  inputs = Input(input_shape)
  base_model = DenseNet121(weights='imagenet', include_top=False,
                          input_tensor=inputs, input_shape=input_shape)
  x = base_model.output
  for layer in base_model.layers:
      layer.trainable = False
  x = Flatten()(x)
  x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
  x = Dropout(0.5)(x)
  predictions = Dense(2, activation='sigmoid')(x)
  model = Model(inputs=inputs, outputs=predictions)
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  history = model.fit(x_train, y_train, batch_size=128, epochs=2, validation_data=(x_test, y_test))
  if history.history['accuracy'][-1] >= min_acc_train and history.history['val_accuracy'][-1]>=min_acc_test:
    break

model.save('drive/MyDrive/cv_pnem/model_densenet121peumonia_1_v2')

Epoch 1/2
61/61 [==============================] - 27s 283ms/step - loss: 1.7259 - accuracy: 0.8928 - val_loss: 0.8191 - val_accuracy: 0.8542
Epoch 2/2
61/61 [==============================] - 8s 127ms/step - loss: 0.4628 - accuracy: 0.9702 - val_loss: 0.6567 - val_accuracy: 0.8782


In [ ]:
model.save('drive/MyDrive/cv_pnem/model_min_DN121peumonia')

In [ ]:
model.evaluate(x_test, y_test)